In [159]:
import numpy as np
import pandas as pd
import re
import openpyxl

In [160]:
power_csv_address=r"C:\Users\jackiehuang\Desktop\power\a.csv"
sar_record_address=r"C:\Users\jackiehuang\Desktop\power\SAR Test Record_20201113 (Version405).xlsm"
output_address=r"test.xlsm"
#raw_stop 要加一
raw_start,raw_stop=25,36

### 讀power csv

In [161]:
df=pd.read_csv(power_csv_address,header=None)

### 顯示表格全部資料(完成可刪)

In [162]:
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

### 清掉空欄位

In [163]:
df.dropna(axis=1,how='all',inplace=True)
df.reset_index(drop=True,inplace=True)

### 加上行名

In [164]:
df.columns=['System Judgement','Band','Test Item', 'Ch', 'Cell Power', 'Lower', 'Upper', 'Unit', 'Result', 'Pass']

### 切出BW, Modulation, RB size, RB offset

In [165]:
def split_Test_Item(n,i):
    pattern=r".*;(.*);(.*)\((\d*)@(\d*)\)"
    m=re.match(pattern,n)
    return m.group(i)

In [166]:
df['BW']=df['Test Item'].apply(split_Test_Item, args=(1,))
df['Modulation']=df['Test Item'].apply(split_Test_Item, args=(2,))
df['RB size']=df['Test Item'].apply(split_Test_Item, args=(3,))
df['RB offset']=df['Test Item'].apply(split_Test_Item, args=(4,))

### 建立df_sar

In [167]:
df_sar= pd.DataFrame()

### 開報告excel

In [168]:
wb=openpyxl.load_workbook(sar_record_address, keep_vba=True)

In [169]:
sheet=wb['SAR']

In [170]:
for i in range(raw_start,raw_stop):
    df_sar=df_sar.append([[sheet[f'B{i}'].value, sheet[f'C{i}'].value, sheet[f'D{i}'].value, sheet[f'E{i}'].value, sheet[f'F{i}'].value, sheet[f'N{i}'].value]])

In [171]:
df_sar.columns=['Band','BW','Modulation','RB size','RB offset','Ch']
df_sar.reset_index(drop=True,inplace=True)

### modulation 切-PRB

In [172]:
def split_modulation(x):
    result=x.split('-')
    return result[0]

In [173]:
df['Modulation']=df['Modulation'].apply(split_modulation)

### 切band

In [174]:
def split_band(x):
    result=x.split()
    return result[2]

In [175]:
df_sar['Band']=df_sar['Band'].apply(split_band)

### BW加HZ

In [176]:
df_sar['BW']=df_sar['BW'].map(lambda x:x+'HZ')

### 合併

In [177]:
df_merge=pd.merge(df_sar.astype(str),df.astype(str),how='left',on=['Band','BW','Modulation','RB size','RB offset','Ch'])

### 填值

In [178]:
for i in range(raw_start,raw_stop):
    sheet[f'R{i}']=df_merge['Result'][i-raw_start]

### 存檔(要3-5minute，我想想)

In [179]:
wb.save(output_address)